# Amazon レビュー分析

## データ収集


In [ ]:
!curl http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Magazine_Subscriptions.json.gz -o Magazine.gz

In [ ]:
!gzip -d Magazine.gz

In [ ]:
!mv Magazine Magazine.json

## EDA(探索的データ解析)

In [ ]:
import pandas as pd
import json

In [ ]:
df_f = pd.read_json('Magazine.json', lines=True)
df_f.head()

In [ ]:
df_f.shape

目的変数として `overall` を取り出す。


In [ ]:
y = df_f.loc[:,['overall']]
y.value_counts()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x='overall',data=df_f)
plt.show()

説明変数として、`vote`(投票), `verified`(認証済み),`reviewTime`(レビューした時間),`reviewerID`(レビュワーID),`asin`(プロダクト ID), `reviewText`(レビューテキスト), `summary`(要約) 


In [ ]:
x = df_f.loc[:,['vote', 'verified','reviewTime', 'reviewerID', 'asin', 'reviewText', 'summary']]
x.head()

In [ ]:
x.describe()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x='reviewerID',data=df_f)
plt.show()

### 前処理

欠損値(`NaN`) の有無を調べる


In [ ]:
y.isnull().sum()

In [ ]:
x.isnull().sum()

今回は、投票の欠損値を `0` で埋める。


In [ ]:
x['vote'] = x['vote'].fillna(0)
x.head()

In [ ]:
review_text = x['reviewText'].values 

wc = WordCloud(
    width=800, 
    height=600,
    background_color='white'
    )

review_wordcloud = wc.generate(str(review_text))

plt.imshow(review_wordcloud)
plt.axis("off")
plt.show()

In [ ]:
summary_text = x['summary'].values 

summary_wordcloud = wc.generate(str(summary_text))

plt.imshow(review_wordcloud)
plt.axis("off")
plt.show()